While there are many public datasets for Reddit, they are often years old or not updated freqently. The goal of the project is to provide information on Covid-19 and related legal advice, so we're dealing with rapidly developing news and would like to have access to recent activity, as well as historical Reddit posts in a specified period of time. The Reddit API (via PRAW as a Python Reddit API Wrapper, making implementation easier) is powerful and can provide a lot of data and metadata, however it is limited in scope for time periods and response size. To address that, we can use the Pushshift API, an API that provides access to a vast amount of public Reddit data that has been ammassed over the years and continues to get updated by the day (? pretty frequenly I'm pretty sure). Another option rather that is accessing the back-end Elasticsearch search engine endpoint (but doesn't work right now?).

Prototype 1: For the moment, Reddit data will use PRAW, which is relatively easy to retrieve. There is an Reddit Api call that can search within a subreddit, which utilizes a cloud-based search engine backend (like Apache Lucene) for internal site searches, but we'd be relying too much on Reddit's relevance algorithms. (There are other Reddit Api functions that get the top/new/hot/controversial posts in a subreddit, but they don't search queries. We could do this, but once again, the Reddit Api is limited in retrieval/time limit, so this would only be for special cases). 
Since to get historic data via Pushshift takes a while and we need to decide how to process a lot of data quickly, we won't use that for now.

In [ ]:
import os
import time
import datetime
import praw
import requests
import datetime
import json
# from wordcloud import WordCloud, STOPWORDS

## Using Pushshift API

Note:
Includes deleted posts...

In [ ]:
# max batch size 1000
# time range field m doesn't work...?
# Retrieving top scored posts from the Coronavirus subreddit from 120 days ago
top = 'https://api.pushshift.io/reddit/search/submission/?subreddit=coronavirus&sort_type=score'

res = requests.get(top)
data_arr = res.json()['data']

In [ ]:
data_arr
for k, obj in enumerate(data_arr):
    print(20*'-', k)
    print('Title:', obj['title'])
    print('Author:', obj['author'])
    print(obj['full_link'])
    print('Score:', obj['score'])
    epoch = obj['created_utc']
    print('Epoch:', epoch)
    print('Date:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch)))   

In [ ]:
print(len(data_arr))

## Using PRAW

In [ ]:
reddit = praw.Reddit(client_id = os.environ['CLIENT_ID'],
                     client_secret = os.environ['CLIENT_SECRET'],
                     user_agent = 'redditdata')

subreddit = reddit.subreddit('all')
top_coronavirus = subreddit.top(limit=None) # 1000 max...?
for k, submission in enumerate(top_coronavirus):
    print(20*'-', k)
    print(submission.title) # The title of the submission.
    print(submission.score) # The number of upvotes for the submission.
#     print(submission.id) # ID of the submission
#     print(submission.permalink) # A permalink for the submission.
#     print(submission.selftext) # The submissions’ selftext - an empty string if a link post.
#     print(submission.url) # The URL the submission links to, or the permalink if a selfpost.
#     print(submission.upvote_ratio) # The percentage of upvotes from all votes on the submission.

    epoch = submission.created_utc
    print('Epoch:', epoch)
    print('Date:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch)))
    # all_reddit.append((submission.title, submission.selftext , submission.id, reddit_base_url + submission.permalink)) 

Using Reddit's search algorithm from its listings (Reddit's data structure for the list of ranked posts.). It utilizes stemming.

In [ ]:
# Default parameters for searching a query in a subreddit.
# query – The query string to search for.
# sort – Can be one of: relevance, hot, top, new, comments. (default: relevance).
# syntax – Can be one of: cloudsearch, lucene, plain (default: lucene).
# time_filter – Can be one of: all, day, hour, month, week, year (default: all).
# TODO: Work on getting other subreddits like CoronavirusNewYork; or filter an extra field, flair=New York, in the Coronavirus subreddit
for k, submission in enumerate(reddit.subreddit('all').search('coronavirus', limit = 1000)):
    print(20*'-', k)
    print(submission.title)
    print(submission.score)
    epoch = submission.created_utc
    print('Epoch:', epoch)
    print('Date:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch)))



In [ ]:
# headers = {'User-agent': 'redditRetrival'}, needed to uniquely identify to prevent rate limiting
res = requests.get('https://www.reddit.com/r/all/search/.json?q=coronavirus&restrict_sr=1&limit=1000', headers = {'User-agent': 'redditRetrival'})
print(res)
vv = res.json()
vv

# Using the 'public' Reddit API, aka through the normal search bar web interface, with .json extension. 

Advantage of this over PRAW is that there is no (rather insignificant) limit rate on requests, as well as request size limit. However, it can only pull 100, regardless of if it is search in subreddit or just the top/hot ranked in a subreddit. (But we probably won't need more than 100?)

https://www.reddit.com/r/all/search/.json?q=coronavirus&restrict_sr=1&limit=1000 \
https://www.reddit.com/r/Coronavirus/top/.json?t=all&limit=1000

In [ ]:
all_data

In [ ]:
def get_submissions_from_pushshift(**kwargs):
    '''
    Use Pushshift API to pull in batches (max 1000).
    
    Use parameters from:
    https://github.com/pushshift/api
    
    By default, results are returned based on created_utc in descending order, but if in ascending order
    if before/after is specified.  Must specify sort_order in that case if want time-window results descending.
    
    parameters:
        subreddit: subreddit to pull results
        after: Return results after this date
        before: Return results before this date
    
    returns:
        list of subreddit data
    '''
    r = requests.get("https://api.pushshift.io/reddit/submission/search/", params=kwargs)
    for i in range(20):
        try:
            data = r.json()
            break
        except:
            print(r)
        time.sleep(2)
    return data['data']

def get_submissions_from_reddit_api(submission_ids):
    '''
    Use the Reddit API to get most updated metadata, such as scores. Useful for most up to data text & scores, 
    but not necessary for data collection
    '''
    headers = {'User-agent':'Submission Collector'}
    params = {}
    params['id'] = ','.join(["t3_" + id for id in submission_ids])
    r = requests.get("https://api.reddit.com/api/info/", params=params, headers=headers)
    data = r.json()
    return data['data']['children']


def get_all_submissions_from_subreddit(subreddit, after=None, before=None):   
    allSubredditData = [] # Collect all data
    
    # Loop in batches
    while True:
        start_time = time.time()
        submissions = get_submissions_from_pushshift(subreddit=subreddit, size=1000, after=after, before=before)
        end_time = time.time()
        print("Retrieved {} results in {:.4f}s. Total results so far: {}".format(len(submissions), end_time - start_time, len(allSubredditData)))
        allSubredditData.extend(submissions)
        
        if not submissions: 
            break

        # This will get the submission ids from Pushshift in batches of 1000 -- Reddit's API only allows 100 at a time
        submission_ids = []
        for submission in submissions:
            after = submission['created_utc'] # This will keep track of your position for the next call in the while loop
            submission_ids.append(submission['id'])
    #     print("first:", submissions[0]['created_utc'])
        print("after:", after)
        epoch = submissions[-1]['created_utc']
#         print('Epoch:', epoch)
        print('Last Date:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch)))

    #     This will then pass the ids collected from Pushshift and query Reddit's API for the most up to date information
    #     submissions = get_submissions_from_reddit_api(submission_ids)
    #     for submission in submissions:
    #         submission = submission['data']
            # Do stuff with the submissions
    #         print(submission['score'], submission['url'], submission['title'])
    #     print()

    #     time.sleep(2) # I'm not sure how often you can query the Reddit API without oauth but once every two seconds should work fine
    
    return allSubredditData

In [ ]:
queries = [{'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "coronavirus"},
           {'before': None, 'after': int(datetime.datetime(year=2019, month=12, day=1).timestamp()), 'subreddit': "legaladvice"}]
for query in queries:
    data = get_all_submissions_from_subreddit(query['subreddit'], before=query['before'], after=query['after'])
    print(len(data))
    print(type(data))
    with open(query['subreddit'] + '.json', 'w') as f:
        json.dump(allSubredditData, f)

In [ ]:
reddit_file = json.load(open('data.json'))
# my_json_string = json.dumps(allSubredditData)

In [ ]:
# my_json_string
reddit_file